### 1. 패키지 설치

In [5]:
!pip install -q langchain langchain-huggingface langchain-community langchain-core langchain-text-splitters bitsandbytes docx2txt langchain-chroma

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=200
)

loader = Docx2txtLoader('./dataset_part_1.docx')
document_list = loader.load_and_split(text_splitter)

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

# 올바른 Hugging Face 모델을 사용한 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

# 확인
print("HuggingFaceEmbeddings initialized successfully!")


/tmp/ipykernel_251705/746791366.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
/home/chae/ch/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


HuggingFaceEmbeddings initialized successfully!


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


# Chroma 데이터베이스 설정
collection_name = 'chroma_art'

# Chroma 데이터베이스 초기화 및 문서 추가
database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory='./chroma_huggingface2'
)

print("Chroma database initialized successfully!")

Chroma database initialized successfully!


In [4]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [5]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = 'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda"  # CUDA에서 자동 배치
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:26<00:00,  3.79s/it]


In [6]:
# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.9,      # 다양성 증가
    top_k=50,             # 상위 k개 토큰 중에서 샘플링
    repetition_penalty=1.03
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_251705/4235283308.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retriever = database.as_retriever(search_kwargs={"k": 2})

/home/chae/ch/lib/python3.12/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [54]:
query = "김기승의 낙양성동도이화의 작품번호는 무엇인가요?"

retrieved_docs = retriever.invoke(query)

In [55]:
retrieved_docs

[Document(metadata={'source': './dataset_part_1.docx'}, page_content='작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n작품 번호: 01370\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1963\n크기: 128×31.5×(6)\n재료: 종이에 먹; 6폭 병풍\n카테고리: 서예\n\n작품 설명'),
 Document(metadata={'source': './dataset_part_1.docx'}, page_content='작품명: 그가 열방사이에- 이사야(2곡병) / N/A / \n작품 번호: 01335\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1972\n크기: 230×60×(2)\n재료: 종이에 먹; 가리개\n카테고리: 서예\n\n작품 설명')]

In [56]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [57]:
ai_message = retrieval_chain.invoke({"input": query})

In [58]:
ai_message

{'input': '김기승의 낙양성동도이화의 작품번호는 무엇인가요?',
 'context': [Document(metadata={'source': './dataset_part_1.docx'}, page_content='작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n작품 번호: 01370\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1963\n크기: 128×31.5×(6)\n재료: 종이에 먹; 6폭 병풍\n카테고리: 서예\n\n작품 설명'),
  Document(metadata={'source': './dataset_part_1.docx'}, page_content='작품명: 그가 열방사이에- 이사야(2곡병) / N/A / \n작품 번호: 01335\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1972\n크기: 230×60×(2)\n재료: 종이에 먹; 가리개\n카테고리: 서예\n\n작품 설명')],
 'answer': 'System: Answer any use questions based solely on the context below:\n\n<context>\n작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n작품 번호: 01370\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1963\n크기: 128×31.5×(6)\n재료: 종이에 먹; 6폭 병풍\n카테고리: 서예\n\n작품 설명\n\n작품명: 그가 열방사이에- 이사야(2곡병) / N/A / \n작품 번호: 01335\n\n\n작가: 김기승 / KIM Kiseung\n\n제작 연도: 1972\n크기: 230×60×(2)\n재료: 종이에 먹; 가리개\n카테고리: 서예\n\n작품 설명\n</context>\nHuman: 김기승의 낙양성동도이화의 작품번호는 무엇인가요?\n\nAI: 김기승의 낙양성의 동향도화(6곡병)의 작품번호는 01370입니다. 이 작품은 1963년에 제작되었으며, 종이에 먹을 사용하여 6폭 병풍

In [59]:
print(ai_message['answer'])

System: Answer any use questions based solely on the context below:

<context>
작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / 
작품 번호: 01370


작가: 김기승 / KIM Kiseung

제작 연도: 1963
크기: 128×31.5×(6)
재료: 종이에 먹; 6폭 병풍
카테고리: 서예

작품 설명

작품명: 그가 열방사이에- 이사야(2곡병) / N/A / 
작품 번호: 01335


작가: 김기승 / KIM Kiseung

제작 연도: 1972
크기: 230×60×(2)
재료: 종이에 먹; 가리개
카테고리: 서예

작품 설명
</context>
Human: 김기승의 낙양성동도이화의 작품번호는 무엇인가요?

AI: 김기승의 낙양성의 동향도화(6곡병)의 작품번호는 01370입니다. 이 작품은 1963년에 제작되었으며, 종이에 먹을 사용하여 6폭 병풍으로 만들어졌습니다. 카테고리는 서예로 분류됩니다.


In [61]:
import re

def extract_question_and_answer(response):
    question = response['input']
    answer_text = response['answer']

    # 정규식을 사용해 AI의 답변 부분만 추출 (AI: 이후의 모든 텍스트)
    match = re.search(r"AI:\s*(.*)", answer_text, re.DOTALL)
    if match:
        answer = match.group(1).strip()
    else:
        answer = "답변을 찾을 수 없습니다."

    return question, answer


In [62]:
question, final_answer = extract_question_and_answer(ai_message)
print(f"질문: {question}")
print('=====================================================')
print(f"{final_answer}")

질문: 김기승의 낙양성동도이화의 작품번호는 무엇인가요?
김기승의 낙양성의 동향도화(6곡병)의 작품번호는 01370입니다. 이 작품은 1963년에 제작되었으며, 종이에 먹을 사용하여 6폭 병풍으로 만들어졌습니다. 카테고리는 서예로 분류됩니다.
